In [1]:
import torch
from utils import print_gpu_memory
print_gpu_memory()
import os
import sys
base_path = r"C:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification"
data_path = r"\data\building-instinct-starter-notebook\Starter notebook"
sys.path.append(base_path+data_path)
sys.path.append(base_path+"\kai")
import pandas as pd
from tqdm import tqdm
# from utils import (calculate_average_hourly_energy_consumption, train_model, get_pred, calculate_hierarchical_f1_score,
# sample_submission_generator)
from preprocessing import Preprocessor
# file_path = base_path + data_path + r"\building-instinct-train-data"
# df_features = Preprocessor.load_standard_df(file_path)
# df_features.sort_index(inplace=True)
# df_features.to_parquet(base_path + '/preprocessed_data/data_9000.parquet', engine='pyarrow')

# df_loaded_1 = pd.read_parquet(base_path + '/preprocessed_data/data_3000.parquet', engine='pyarrow')
# df_loaded_2 = pd.read_parquet(base_path + '/preprocessed_data/data_6000.parquet', engine='pyarrow')
# df_loaded_3 = pd.read_parquet(base_path + '/preprocessed_data/data_9000.parquet', engine='pyarrow')

# df_features = pd.concat([df_loaded_1, df_loaded_2, df_loaded_3], axis=0)
# df_features.sort_index(inplace=True)
# df_features.to_parquet(base_path + '/preprocessed_data/standard_data.parquet', engine='pyarrow')

df_features = pd.read_parquet(base_path + '/preprocessed_data/standard_data.parquet', engine='pyarrow')
df_features.sort_index(inplace=True)

Allocated memory: 0.00 GB
Cached memory: 0.00 GB
Total memory: 12.00 GB
Unused memory: 12.00 GB


In [2]:
from preprocessing import TargetPreprocessor
load_filepath_labels = os.path.join(base_path + data_path,'building-instinct-train-label', 'train_label.parquet')#path to the train label file
df_targets = pd.read_parquet(load_filepath_labels, engine='pyarrow')
df_targets.sort_index(inplace=True)

df_targets_res = df_targets[df_targets.building_stock_type == "residential"].filter(like='_res').copy()
df_targets_com = df_targets[df_targets.building_stock_type == "commercial"].filter(like='_com').copy()
target_preprocessor = TargetPreprocessor()
df_targets_res, association_dict_res, encoder_res = target_preprocessor.preprocess_res(df_targets_res)
df_targets_com, association_dict_com, encoder_com = target_preprocessor.preprocess_com(df_targets_com)

In [3]:
common_indices = df_features.index.intersection(df_targets_com.index)
# Filter the data
X_com = df_features[df_features.index.isin(common_indices)]
X_com = torch.tensor(X_com.values, dtype=torch.float32)
y_com = df_targets_com[df_targets_com.index.isin(common_indices)]
y_com = torch.tensor(y_com.values, dtype=torch.float32)

X_res = df_features[df_features.index.isin(df_targets_res.index)]
X_res = torch.tensor(X_res.values, dtype=torch.float32)
y_res = df_targets_res[df_targets_res.index.isin(df_features.index)]
y_res = torch.tensor(y_res.values, dtype=torch.float32)

## classifier

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt

X, y = df_features, df_targets["building_stock_type"].map({"residential": 0, "commercial": 1})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = Pipeline([('preprocessor', ColumnTransformer([
            ('scaler', StandardScaler(), X.columns),
            ('encoder', OneHotEncoder(), [])
        ])),
        ('classifier', RandomForestClassifier(random_state=42))
    ])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f1_score(y_test, y_pred, average='macro'))
print(confusion_matrix(y_test, y_pred))

0.992323346090601
[[765   2]
 [  9 664]]


In [5]:
df_test = pd.read_parquet(base_path + '/preprocessed_data/data_test.parquet', engine='pyarrow')
df_test.sort_index(inplace=True)

clf = Pipeline([('preprocessor', ColumnTransformer([
            ('scaler', StandardScaler(), X.columns),
            ('encoder', OneHotEncoder(), [])
        ])),
        ('classifier', RandomForestClassifier(random_state=42))
    ])
clf.fit(X, y)
y_pred = clf.predict(df_test)
df_test["building_stock_type"] = y_pred

## com model

In [6]:
# from torch.utils.data import DataLoader, Dataset
# from models import MultiTaskLSTM, CustomLoss, TimeSeriesDataset
# from datetime import datetime

# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# torch.cuda.empty_cache()

# # model parameters
# batch_size = 16
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# input_size, sequence_length = 1, X_com.shape[1]
# num_classes_categorical = y_com.shape[1]
# hidden_size = 64
# num_epochs = 20

# weight_numerical = 10e-4
# weight_categorical = 1.0

# best_combined_loss = float('inf')
# checkpoint_filename = base_path+ f'/kai/checkpoints/model_checkpoint_{datetime.now().strftime("%m_%d_%H_%M")}.pth.tar'

# # create dataloaders
# dataloader_com = DataLoader(TimeSeriesDataset(X_com, y_com), batch_size=batch_size, shuffle=True)
# dataloader_res = DataLoader(TimeSeriesDataset(X_res, y_res), batch_size=batch_size, shuffle=True)

# model = MultiTaskLSTM(input_size, hidden_size, num_classes_categorical)
# model = model.to(device)
# criterion = CustomLoss(association_dict_com)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# for epoch in range(num_epochs):
#     model.train()
#     epoch_loss = 0.0
#     for X_batch, y_categorical_batch in dataloader_com:
#         X_batch = X_batch.view(X_batch.shape[0], sequence_length, input_size)
#         X_batch = X_batch.to(device)
#         y_categorical_batch = y_categorical_batch.to(device)
#         optimizer.zero_grad()
#         categorical_pred = model(X_batch)
#         loss, loss_numerical, loss_categorical = criterion(categorical_pred, y_categorical_batch)
        
#         # Combine losses with adjusted weights
#         combined_loss = weight_numerical * loss_numerical + weight_categorical * loss_categorical

#         combined_loss.backward()
#         optimizer.step()
#         epoch_loss += loss.item()
#     print(f'Epoch [{epoch+1}/{num_epochs}], '
#                   f'Total Loss: {loss.item():.4f}, '
#                   f'Numerical Loss: {loss_numerical.item():.4f}, '
#                   f'Categorical Loss: {loss_categorical.item():.4f}, '
#                   f'Combined Loss: {combined_loss.item():.4f}, '
#                   f'weights: {weight_numerical:.4f}, {weight_categorical:.4f}')
    
#     if combined_loss.item() < best_combined_loss:
#         torch.save({'epoch': epoch + 1,
#                     'model_state_dict': model.state_dict(),
#                     'optimizer_state_dict': optimizer.state_dict(),
#                     'loss': combined_loss.item(),}, checkpoint_filename)


In [7]:
from torch.utils.data import DataLoader, Dataset
from models import MultiTaskLSTM, CustomLoss, TimeSeriesDataset
from datetime import datetime

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
batch_size, input_size, sequence_length, hidden_size, num_classes_categorical = 16, 1, X_com.shape[1], 64, y_com.shape[1]
checkpoint_filename = base_path+ f'/kai/checkpoints/com_model_checkpoint_08_06_14_49.pth.tar'

dataloader_com = DataLoader(TimeSeriesDataset(X_com, y_com), batch_size=batch_size, shuffle=True)
model = MultiTaskLSTM(input_size, hidden_size, num_classes_categorical)
model = model.to(device)

checkpoint = torch.load(checkpoint_filename)
model.load_state_dict(checkpoint['model_state_dict'])
for X_batch, y_categorical_batch in dataloader_com:
    for i in [0, 1]:
        _X_batch = X_batch[:, :].view(X_batch.shape[0], sequence_length, input_size)
        _X_batch = _X_batch.to(device)
        categorical_pred = model.predict(_X_batch, association_dict_com)
        print(categorical_pred)
        print(y_categorical_batch[i, :])
        break
    break

C:\Users\KAI\AppData\Local\Temp\ipykernel_4584\3095246467.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_filename)


tensor([[2.2376e+00, 1.9897e+03, 3.5270e+02, 3.4938e+02, 8.7198e+00, 1.8647e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00],
        [2.2700e+00, 1.9236e+03, 3.4095e+02, 3.3733e+02, 8.6536e+00, 1.8160e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00],
        [2.2806e+00, 1.9351e+03, 3.4299e+02, 3.3937e+02, 8.6881e+00, 1.8266e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+

### create the original com df

In [8]:
from postprocessing import inverse_process
arr_df = inverse_process(categorical_pred, encoder_com) # todo: standardize the target numerical columns in preprocessing
display(arr_df)

,in.number_of_stories_com,in.vintage_com,in.tstat_clg_sp_f..f_com,in.tstat_htg_sp_f..f_com,in.weekday_opening_time..hr_com,in.weekday_operating_hours..hr_com,in.comstock_building_type_group_com,in.heating_fuel_com,in.hvac_category_com,in.ownership_type_com,in.wall_construction_type_com
0,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
1,2,Before 1946,80,72,8.75,18.25,Office,Electricity,Small Packaged Unit,leased,SteelFramed
2,2,Before 1946,80,72,8.75,18.25,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
3,2,1970 to 1979,80,72,8.75,18.5,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
4,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
5,2,Before 1946,80,72,8.75,18.25,Office,Electricity,Small Packaged Unit,leased,SteelFramed
6,2,Before 1946,80,72,8.75,18.25,Office,Electricity,Small Packaged Unit,leased,SteelFramed
7,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
8,2,1980 to 1989,80,72,8.75,18.75,Mercantile,Electricity,Small Packaged Unit,leased,SteelFramed
9,2,Before 1946,80,72,8.75,18.0,Office,Electricity,Small Packaged Unit,leased,SteelFramed


In [9]:
# file_path = base_path + data_path + r"\building-instinct-test-data"
# df_features = Preprocessor.load_standard_df(file_path)
# df_features.sort_index(inplace=True)
# df_features.to_parquet(base_path + '/preprocessed_data/data_test.parquet', engine='pyarrow')

# df_test = pd.read_parquet(base_path + '/preprocessed_data/data_test.parquet', engine='pyarrow')
# df_test.sort_index(inplace=True)

# TODO: clf that predicts com or res
# then filter by com & res
df_test_com = df_test[df_test.building_stock_type == 1]
# df_test = df_test[df_test.index.isin(df_features.index)]
# X_test_com = torch.tensor(df_test.values, dtype=torch.float32)
# X_test_com = X_test_com.to(device)
print(df_test_com[df_test_com.columns.difference(["building_stock_type"])].shape)
X_test_com, y_pred_com = torch.tensor(df_test_com[df_test_com.columns.difference(["building_stock_type"])].values, dtype=torch.float32), df_test_com["building_stock_type"]

(654, 35040)


In [10]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
def free_gpu_memory():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
free_gpu_memory()
model = MultiTaskLSTM(input_size, hidden_size, num_classes_categorical)
model = model.to(device)
criterion = CustomLoss(association_dict_com)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
checkpoint_filename = base_path+ f'/kai/checkpoints/com_model_checkpoint_08_06_14_49.pth.tar'
checkpoint = torch.load(checkpoint_filename)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
best_loss = checkpoint['loss']

predictions = []
dataloader = DataLoader(TimeSeriesDataset(X_test_com, torch.zeros(X_test_com.shape[0], y_com.shape[1])), batch_size=16, shuffle=False)

print_gpu_memory()
for X_batch, y_categorical_batch in tqdm(dataloader):
    model.eval()
    _X_batch = X_batch.view(X_batch.shape[0], sequence_length, input_size)
    _X_batch = _X_batch.to(device)
    with torch.no_grad():
        categorical_pred = model.predict(_X_batch, association_dict_com)
    predictions.append(categorical_pred.cpu())
    del _X_batch, categorical_pred
    free_gpu_memory()
predictions = torch.cat(predictions, dim=0)
arr_df_com = inverse_process(predictions, encoder_com)
arr_df_com.index = df_test_com.index

C:\Users\KAI\AppData\Local\Temp\ipykernel_4584\3946375991.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_filename)


Allocated memory: 3.22 GB
Cached memory: 3.23 GB
Total memory: 12.00 GB
Unused memory: 8.78 GB


100%|██████████| 41/41 [00:11<00:00,  3.64it/s]


In [11]:
for col in arr_df_com.columns:
    print(arr_df_com[col].unique())

[2]
['1980 to 1989' '1960 to 1969' 'Before 1946' '1970 to 1979' '1946 to 1959']
[80]
[72]
[8.75]
[18.75 18.5 18.25 18.0]
['Mercantile' 'Office']
['Electricity']
['Small Packaged Unit']
['leased']
['SteelFramed']


## res

In [12]:
print(X_res.shape, y_res.shape)
print(association_dict_res)
print(association_dict_com)
print(y_res[:, [6, 7, 8, 9, 10, 11, 12]])

torch.Size([3900, 35040]) torch.Size([3900, 38])
{'in.geometry_building_type_recs': [6, 7, 8, 9, 10], 'in.geometry_foundation_type': [11, 12, 13, 14, 15, 16], 'in.geometry_wall_type': [17, 18, 19, 20], 'in.heating_fuel': [21, 22, 23, 24, 25, 26], 'in.roof_material': [27, 28, 29, 30, 31, 32, 33], 'in.tenure': [34, 35], 'in.vacancy_status': [36, 37]}
{'in.comstock_building_type_group': [6, 7, 8, 9, 10, 11, 12], 'in.heating_fuel': [13, 14, 15, 16, 17], 'in.hvac_category': [18, 19, 20, 21, 22], 'in.ownership_type': [23, 24, 25, 26, 27], 'in.wall_construction_type': [28, 29, 30, 31]}
tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define custom loss function
class CustomLoss_res(nn.Module):
    def __init__(self, column_groups):
        super(CustomLoss_res, self).__init__()
        self.mse_loss = nn.MSELoss()
        self.column_groups = column_groups  # Dictionary mapping attribute prefixes to column indices
        self.end_numerical = int(min([min(v) for v in self.column_groups.values()]))
    
    def forward(self, categorical_pred, categorical_true):
        # Compute numerical loss (assuming the first few columns are numerical)
        loss_numerical = self.mse_loss(categorical_pred[:, :self.end_numerical], categorical_true[:, :self.end_numerical])

        # Initialize categorical loss
        loss_categorical = 0.0
        
        # For each attribute group, compute the cross-entropy loss
        for attr, indices in self.column_groups.items():
            # Extract logits for the current attribute
            logits = categorical_pred[:, indices]
            
            # Extract the true labels for the current attribute
            # Convert one-hot encoding to class indices
            true_labels = torch.argmax(categorical_true[:, indices], dim=1)
            
            # Compute cross-entropy loss for the current attribute
            loss_categorical += F.cross_entropy(logits, true_labels)

        total_loss = loss_numerical + loss_categorical
        return total_loss, loss_numerical, loss_categorical   

In [14]:
# from torch.utils.data import DataLoader, Dataset
# from models import MultiTaskLSTM, CustomLoss, TimeSeriesDataset
# from datetime import datetime

# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# torch.cuda.empty_cache()

# # model parameters
# batch_size = 16
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# input_size, sequence_length = 1, X_res.shape[1]
# num_classes_categorical = y_res.shape[1]
# hidden_size = 64
# num_epochs = 20

# weight_numerical = 10e-4
# weight_categorical = 1.0

# best_combined_loss = float('inf')
# checkpoint_filename = base_path+ f'/kai/checkpoints/res_model_checkpoint_{datetime.now().strftime("%m_%d_%H_%M")}.pth.tar'

# # create dataloaders
# # dataloader_com = DataLoader(TimeSeriesDataset(X_com, y_com), batch_size=batch_size, shuffle=True)
# dataloader_res = DataLoader(TimeSeriesDataset(X_res, y_res), batch_size=batch_size, shuffle=True)

# model = MultiTaskLSTM(input_size, hidden_size, num_classes_categorical)
# model = model.to(device)
# criterion_res = CustomLoss_res(association_dict_res)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# for epoch in range(num_epochs):
#     model.train()
#     epoch_loss = 0.0
#     for X_batch, y_categorical_batch in dataloader_res:
#         X_batch = X_batch.view(X_batch.shape[0], sequence_length, input_size)
#         X_batch = X_batch.to(device)
#         y_categorical_batch = y_categorical_batch.to(device)
#         optimizer.zero_grad()
#         categorical_pred = model(X_batch)
#         loss, loss_numerical, loss_categorical = criterion_res(categorical_pred, y_categorical_batch)
        
#         # Combine losses with adjusted weights
#         combined_loss = weight_numerical * loss_numerical + weight_categorical * loss_categorical

#         combined_loss.backward()
#         optimizer.step()
#         epoch_loss += loss.item()
#     print(f'Epoch [{epoch+1}/{num_epochs}], '
#                   f'Total Loss: {loss.item():.4f}, '
#                   f'Numerical Loss: {loss_numerical.item():.4f}, '
#                   f'Categorical Loss: {loss_categorical.item():.4f}, '
#                   f'Combined Loss: {combined_loss.item():.4f}, '
#                   f'weights: {weight_numerical:.4f}, {weight_categorical:.4f}')
    
#     if combined_loss.item() < best_combined_loss:
#         torch.save({'epoch': epoch + 1,
#                     'model_state_dict': model.state_dict(),
#                     'optimizer_state_dict': optimizer.state_dict(),
#                     'loss': combined_loss.item(),}, checkpoint_filename)

### todo postprocessing + analysis of res and com labels in general! 

In [15]:
def map_to_closest_values(predictions, possible_values):
    """
    Map predicted values to the closest possible values.

    Args:
    - predictions (list of lists): The predicted values.
    - possible_values (dict): A dictionary where keys are column names and values are lists of possible values.

    Returns:
    - mapped_predictions (list of lists): The predictions mapped to the closest possible values.
    """
    def closest_value(predicted, possible):
        return possible[np.argmin((np.array(possible) - predicted)**2)]
    
    mapped_predictions = []
    
    for prediction in predictions:
        mapped_row = []
        for col_name, pred_value in zip(possible_values.keys(), prediction):
            mapped_value = closest_value(pred_value, possible_values[col_name])
            mapped_row.append(mapped_value)
        mapped_predictions.append(mapped_row)
    
    return mapped_predictions

def inverse_process_res(prediction, encoder):
    prediction = prediction.cpu().detach().numpy()

    # Possible values for each column
    possible_values = {
        'in.bedrooms_res': [3, 2, 1, 4, 5],
        'in.cooling_setpoint_res': [68, 75, 72, 70, 78, 80, 60, 65, 76, 67, 62],
        'in.heating_setpoint_res': [75, 72, 68, 70, 62, 55, 65, 78, 76, 67, 60, 80],
        'in.geometry_floor_area_res': [1749,  874, 1249,  624, 2249, 4000, 2749, 3499,  249],
        'in.income_res': [109999,  12499,  64999,  89999,  42499,  22499,  47499, 169999, 129999,
                            200000,  37499,  54999,  32499,  10000, 189999,  74999,  17499,  27499,
                            149999],
        'in.vintage_res': [1940, 1975, 1985, 1945, 1995, 1955, 1965, 2005, 2015]
    }
    mapped_predictions = prediction.copy()
    # Map the predictions
    mapped_predictions_1 = map_to_closest_values(prediction[:, :6], possible_values)
    mapped_predictions_2 = encoder.inverse_transform(prediction[:, 6:])

    mapped_predictions = np.concatenate((mapped_predictions_1, mapped_predictions_2), axis=1)
    cols = ['in.bedrooms_res', 'in.cooling_setpoint_res', 'in.heating_setpoint_res',
            'in.geometry_floor_area_res', 'in.income_res', 'in.vintage_res',
            'in.geometry_building_type_recs_res', 'in.geometry_foundation_type_res', 
            'in.geometry_wall_type_res', 'in.heating_fuel_res', 'in.roof_material_res',
            'in.tenure_res', 'in.vacancy_status_res']
    
    mapped_df = pd.DataFrame(mapped_predictions, columns=cols)

    vintage_mapping = {1940:'<1940', 1945:'1940s', 1955:'1950s', 1965:'1960s',
                           1975:'1970s', 1985:'1980s', 1995:'1990s', 2005:'2000s', 2015:'2010s'}
    mapped_df['in.vintage_res'] = mapped_df['in.vintage_res'].map(vintage_mapping)
    mapped_df['in.cooling_setpoint_res'] = mapped_df['in.cooling_setpoint_res'].apply(lambda x: str(x)+"F").astype(str)
    mapped_df['in.heating_setpoint_res'] = mapped_df['in.heating_setpoint_res'].apply(lambda x: str(x)+"F").astype(str)
    mapped_df['in.bedrooms_res'] = mapped_df['in.bedrooms_res'].astype(str)

    income_mapping = {109999: '100000-119999', 12499: '10000-14999', 64999: '60000-69999', 89999: '80000-99999', 
                      42499: '40000-44999', 22499: '20000-24999', 47499: '45000-49999', 169999: '160000-179999', 
                      129999: '120000-139999', 200000: '200000+', 37499: '35000-39999', 54999: '50000-59999', 
                      32499: '30000-34999', 10000: '<10000', 189999: '180000-199999', 74999: '70000-79999', 
                      17499: '15000-19999', 27499: '25000-29999', 149999: '140000-159999'}
    mapped_df['in.income_res'] = mapped_df['in.income_res'].map(income_mapping)

    geometry_mapping = {1749: '1500-1999', 874: '750-999', 1249: '1000-1499', 624: '500-749', 2249: '2000-2499',
                        4000: '4000+', 2749: '2500-2999', 3499: '3000-3999', 249: '0-499'}
    mapped_df['in.geometry_floor_area_res'] = mapped_df['in.geometry_floor_area_res'].map(geometry_mapping)

    return mapped_df

In [19]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
def free_gpu_memory():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
free_gpu_memory()

batch_size = 16
device = "cuda:0" if torch.cuda.is_available() else "cpu"
input_size, sequence_length = 1, X_res.shape[1]
num_classes_categorical = y_res.shape[1]
hidden_size = 64
num_epochs = 20
model = MultiTaskLSTM(input_size, hidden_size, num_classes_categorical)
model = model.to(device)
criterion = CustomLoss_res(association_dict_res)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
checkpoint_filename = base_path+ f'/kai/checkpoints/res_model_checkpoint_08_09_22_35.pth.tar'
checkpoint = torch.load(checkpoint_filename)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
best_loss = checkpoint['loss']

predictions = []

df_test_res = df_test[df_test.building_stock_type == 0]
X_test_res, y_pred_res = torch.tensor(df_test_res[df_test_res.columns.difference(["building_stock_type"])].values, dtype=torch.float32), df_test_res["building_stock_type"]
dataloader = DataLoader(TimeSeriesDataset(X_test_res, torch.zeros(X_test_res.shape[0], y_res.shape[1])), batch_size=16, shuffle=False)

print_gpu_memory()
for X_batch, y_categorical_batch in tqdm(dataloader):
    model.eval()
    _X_batch = X_batch.view(X_batch.shape[0], sequence_length, input_size)
    _X_batch = _X_batch.to(device)
    with torch.no_grad():
        categorical_pred = model.predict(_X_batch, association_dict_res)
    predictions.append(categorical_pred.cpu())
    del _X_batch, categorical_pred
    free_gpu_memory()
predictions = torch.cat(predictions, dim=0)
arr_df_res = inverse_process_res(predictions, encoder_res)
arr_df_res.index = df_test_res.index

C:\Users\KAI\AppData\Local\Temp\ipykernel_4584\3493477853.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_filename)


Allocated memory: 0.01 GB
Cached memory: 0.02 GB
Total memory: 12.00 GB
Unused memory: 11.99 GB


100%|██████████| 50/50 [00:13<00:00,  3.61it/s]


In [20]:
for col in arr_df_res.columns:
    print(col)
    print(arr_df_res[col].unique())

in.bedrooms_res
['4']
in.cooling_setpoint_res
['72F']
in.heating_setpoint_res
['67F']
in.geometry_floor_area_res
['1500-1999']
in.income_res
['70000-79999']
in.vintage_res
['1980s']
in.geometry_building_type_recs_res
['Single-Family Detached']
in.geometry_foundation_type_res
['Slab']
in.geometry_wall_type_res
['Wood Frame']
in.heating_fuel_res
['Natural Gas']
in.roof_material_res
['Composition Shingles']
in.tenure_res
['Renter']
in.vacancy_status_res
['Occupied']


## create the submission

In [23]:
def create_submission(df_com, df_res, df_test, save_filepath=None):

    load_filepath_labels = os.path.join(base_path + data_path,'building-instinct-train-label', 'train_label.parquet')

    df_targets = pd.read_parquet(load_filepath_labels, engine='pyarrow')
    df_targets.sort_index(inplace=True)

    bldg_id_list = [i for i in range(1,1441)]

    df = pd.DataFrame(index=bldg_id_list, columns=df_targets.columns)
    df.index.name = df_targets.index.name

    # Populate the first column 'building_stock_type'
    df['building_stock_type'] = df_test["building_stock_type"].map({0: 'residential', 1: 'commercial'})

    # Separate columns into residential and commercial
    res_columns = [col for col in df_targets.columns if col.endswith('_res')]
    com_columns = [col for col in df_targets.columns if col.endswith('_com')]

    # Populate the rest of the columns based on the value of 'building_stock_type'
    for bldg_id in df.index:
        if df.at[bldg_id, 'building_stock_type'] == 'residential':
            df.loc[bldg_id, com_columns] = np.nan
            for col in res_columns:
                df.at[bldg_id, col] = df_res.at[bldg_id, col]
        else:  # commercial
            df.loc[bldg_id, res_columns] = np.nan
            for col in com_columns:
                df.at[bldg_id, col] = df_com.at[bldg_id, col]
    df = df.astype(str)
    if save_filepath:
        df.to_parquet(save_filepath)
    return df

df = create_submission(arr_df_com, arr_df_res, df_test, save_filepath="submission.parquet")
print(df)

        building_stock_type in.comstock_building_type_group_com  \
bldg_id                                                           
1               residential                                 nan   
2               residential                                 nan   
3                commercial                          Mercantile   
4                commercial                          Mercantile   
5               residential                                 nan   
...                     ...                                 ...   
1436             commercial                          Mercantile   
1437            residential                                 nan   
1438            residential                                 nan   
1439             commercial                          Mercantile   
1440            residential                                 nan   

        in.heating_fuel_com in.hvac_category_com in.number_of_stories_com  \
bldg_id                                            